# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder. Additionally, Make sure to delete or comment out any code you add to this notebook for testing purposes prior to submitting the notebook to Gradescope. Failure to do so may crash the autograder. 
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You should not use - and do not need - any helper functions; implement the required functions by completing the function skeletons below. All of the fuctions you implement should be self-contained; each function should carry all of the code it needs to execute on its expected input(s) within its body. 

### Kernel Selection

Be sure that you select the PySpark kernel from the kernel options above, if it is not already selected. Do not use the Python3 kernel. The notebook should default to the PySpark kernel. 

### GCP and Gradescope
For submitting to Gradescope, you should use spark dataframes and dataframe operations _only_. 
You should not use SQL. 

### SparkSession Objects and SparkContexts
Note that you have access to a SparkSession object in the notebook environment in virtue of using the PySpark kernel. It can be referenced using the variable name _spark_ and does not need to be explicitly created or defined. This beavior can be leveraged to read in the initial dataset. 

You should not need to invoke _spark_ in any of your functions but the load_data() function. Do not hardcode references to _spark_ into any of your functions except for the load_data() function. 

### Assumptions Regarding Function Inputs
This question assumes that you're carrying the results of your prior work foward at each stage of data preparation and analysis. Thus, the _input_ to each of the _filtering_ functions below should be the _output_ of the _prior_ function. The inputs to the two analytical functions should be the output of the last filtering function. 

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [1]:
import pyspark

In [2]:
### Student Section - Please compete all the functions below

#### Function to return GT Username

In [3]:
#export
def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'yyang3041'

#### Function to load data

In [4]:
#export
def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP  #
    # storage bucket                                               #      
    ################################################################
    df = spark.read.csv(gcp_storage_path, header=True, inferSchema=True)
    return df

#### Function to exclude trips that don't have a pickup location

In [5]:
#export
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to exclude trips with no pickup locations               #
    # Note: Exclude nulls and zeros                                #        
    ################################################################
    dfo = df.filter((df["PULocationID"].isNotNull()) & (df['PULocationID'] != 0))
    return dfo

#### Function to exclude trips with no distance

In [6]:
#export
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to exclude trips with no trip distances                 #
    # Note: Exclude nulls and zeros                                #        
    ################################################################
    dfo = df.filter((df["trip_distance"].isNotNull()) & (df['trip_distance'] != 0))
    dfo = dfo.withColumn("trip_distance", dfo["trip_distance"].cast('decimal(38,10)'))
    return dfo

#### Function to include fare amount between the range of 20 to 60 Dollars

In [7]:
#export
def include_fare_range(df):
    
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars (including 20 and 60 dollars)               #        
    ################################################################
    dfo = df.filter(df['fare_amount'].between(20,60))
    dfo = dfo.withColumn("fare_amount", dfo["fare_amount"].cast('decimal(38,10)'))
    return dfo

#### Function to get the highest tip amount

In [8]:
#export
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get the highest tip amount                           #
    #                                                              #        
    ################################################################
    newdf = df.withColumn("tip_amount", df["tip_amount"].cast('decimal(38,10)'))
    newdf = newdf.orderBy(newdf["tip_amount"].desc()).limit(1)
    ret = newdf.select("tip_amount").collect()[0][0]
    return round(ret, 2)

#### Function to get total toll amount

In [9]:
#export
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get total toll amount                                #
    #                                                              #        
    ################################################################
    dfo = df.withColumn("tolls_amount", df["tolls_amount"].cast('decimal(38,10)'))
    ret = dfo.agg({"tolls_amount": "sum"}).collect()[0][0]
    return round(ret, 2)

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata09-08-2021.csv

In [10]:
gcp_storage_path = "gs://yyang3041/yellow_tripdata09-08-2021.csv"
df = load_data(gcp_storage_path)
df.printSchema()

root
 |-- vendorid: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- ratecodeid: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pulocationid: integer (nullable = true)
 |-- dolocationid: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



#### Print total numbers of rows in the dataframe

In [11]:
df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [12]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()

3833896

#### Print total number of rows in the dataframe after exclude trips with no distance

In [13]:
df_no_trip_distance = exclude_no_tripdistance(df_no_pickup_locations)
df_no_trip_distance.count()

2030274

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [14]:
df_include_fare_range = include_fare_range(df_no_trip_distance)
df_include_fare_range.count()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|vendorid|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|store_and_fwd_flag|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:45:21|  2019-01-01 01:31:05|              1|          8.7|         1|                 N|         163|          25|           1|       34.5|  0.5|    0.5|      7.15|         0.0|                  0.3

255321

#### Print the highest tip amount

In [15]:
max_tip = get_highest_tip(df_include_fare_range)
print(max_tip)

212.00


#### Print the total toll amount

In [16]:
total_toll = get_total_toll(df_include_fare_range)
print(total_toll)

561387.86
